<a href="https://colab.research.google.com/github/amalkoodoruth/deep-learning-for-pkd-patients/blob/main/new_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting drive and changing the working directory
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MiCM2021-PKD/dataloader_data_kaggle/Train_Sets/MR

Mounted at /content/drive
/content/drive/MyDrive/MiCM2021-PKD/dataloader_data_kaggle/Train_Sets/MR


In [2]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 29.6MB/s 


In [ ]:
!pip install PIL

In [3]:
# importing important librarires
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pydicom
from PIL import Image

In [4]:
my_dir = !pwd
my_dir = str(my_dir)

In [6]:
!pip install elasticdeform

     |████████████████████████████████| 102kB 9.1MB/s 


In [7]:
import numpy, imageio, elasticdeform
X = numpy.zeros((200, 300))
X[::10, ::10] = 1

# apply deformation with a random 3 x 3 grid
X_deformed = elasticdeform.deform_random_grid(X, sigma=25, points=3)

imageio.imsave('test_X.png', X)
imageio.imsave('test_X_deformed.png', X_deformed)

In [8]:
ls

1/   13/  19/  20/  22/  31/  33/  36/  38/  5/  test_X_deformed.png
10/  15/  2/   21/  3/   32/  34/  37/  39/  8/  test_X.png


In [9]:
import torch
from torch.utils import data
from torch.autograd import Variable

import sys

In [16]:
patients = os.listdir(my_dir[2:-2])
image_paths = []
for patient in patients:
  image_path = my_dir[2:-2] + '/' + patient
  image_paths.append(image_path)


In [17]:
print(image_paths[0])

/content/drive/My Drive/MiCM2021-PKD/dataloader_data_kaggle/Train_Sets/MR/8


In [ ]:
my_data_dir = data_dir + '13/T2SPIR/Ground/'
master_dir = '/content/drive/My Drive/MiCM2021-PKD/dataloader_data_kaggle/Train_Sets/MR/'
patients = os.listdir(master_dir)
#patients.sort()
X_train_paths = []
y_train_paths = []
for patient in patients:
  X_train_paths.append(master_dir + patient + '/T2SPIR/dicom_anon')
  y_train_paths.append(master_dir + patient + '/T2SPIR/Ground')




In [ ]:
class SliceDatasetDeformable(data.Dataset):

    ##
    # img_paths is list of paths to intensity images
    # seg_paths is list of paths to segmentation images, define as None if no segmentations exist
    # sigma is deformation intensity, points the number of coordinates for grid deformation
    def __init__(self, img_paths, seg_paths, sigma, points, new_dimensions):

        self.seg_paths = seg_paths
        self.img_paths = img_paths
        self.sigma = sigma
        self.points = points
        self.new_dimensions = new_dimensions

    def __len__(self):
        return len(self.img_paths)


    def __getitem__(self, index):
        """
        This function is used to retrieve one scan and its corresponding segmented
        image from a dataset, if it exists. The scan and the segmented image are 
        converted into numpy arrays that have the dimensions specified in the constructor.

          Parameters:
            index (int): the index of the scan we want to retrieve. It is in the range [0, size of dataset)

          Returns:
            X (numpy.ndarray): training sample of size new_dimensions
            Y (numpy.ndarray): segmented image if it exists (will exist if in training set). Else, array of 0s
              
        """
        image_path = self.image_paths[index]
        image = np.load_scan(image_path)
        image_resized = resize_data(image, new_dimensions)

        seg_exists = len(self.seg_paths) > 0

        if seg_exists:
          seg_path = self.seg_paths[index]
          seg = np.load_seg(seg_path)
          seg_resized = resize_data(seg, new_dimensions)

        # Convert images to pytorch tensors
        ## why .float() and .long()
        X = Variable(torch.from_numpy(image_resized)).float()
    
        if seg_exists:
            Y = Variable(torch.from_numpy(seg_resized)).long()

        else:
            Y = torch.zeros(1) # dummy segmentation
          
        name = os.path.basename(self.img_paths[index])

        return X, Y, name ## why return name??



    # Load the scans in given folder path
    def load_scan(self, path):
        """
        This function is used to load the MRI scans. It converts the scan into a numpy array

          Parameters:
            path (str): The path to the folder containing the MRI scans of all patients

          Returns:
            np_image (numpy.ndarray): A numpy array representing the MRI scan
        """

        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        try:
            slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
        except Exception as e:
            print("Exception raised: ", e)
            slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
            
        for s in slices:
            s.SliceThickness = slice_thickness

        image = np.stack([s.pixel_array for s in slices])
        image = image.astype(np.int16)
        np_image = np.array(image, dtype=np.int16)

        return np_image
            
    
    def load_seg(self,path):
        """
        This function is used to load the segmented image. It returns the image in a numpy array

          Parameters:
            path (str): The directory where all the segmented images corresponding to one patient are stored

          Returns:
            seg_data (numpy.ndarray): A list of numpy arrays corresponding to segmented images
        """
        seg_paths = []

        if path[-1] != '/':
          path = path + '/'

        for seg in os.listdir(path):
          seg_paths.append(path + seg)

        seg_paths.sort()

        seg_data = []
        for seg_path in seg_paths:
          seg = Image.open(seg_path)
          seg_data.append(np.asarray(seg))

        return seg_data

        
    def resize_data(self, data, new_dimensions):
        '''
        This function resizes a numpy array.
        TO DO: method used for interpolation?

          Parameters:
            data (numpy.ndarray): a numpy array representing an MRI scan
            new_dimensions (list): a list containing the dimensions of the new scan [z,x,y]

          Returns:
            new_data (numpy.ndarray): a numpy array with the desired dimensions
        '''
        initial_size_x = data.shape[1]
        initial_size_y = data.shape[2]
        initial_size_z = data.shape[0]

        new_size_z = new_dimensions[0]
        new_size_x = new_dimensions[1]
        new_size_y = new_dimensions[2]
          
        delta_x = initial_size_x / new_size_x
        delta_y = initial_size_y / new_size_y
        delta_z = initial_size_z / new_size_z

        new_data = np.zeros((new_size_z, new_size_x, new_size_y))

        for x, y, z in itertools.product(range(new_size_x),
                                          range(new_size_y),
                                          range(new_size_z)):
          new_data[z][x][y] = data[int(z * delta_z)][int(x * delta_x)][int(y * delta_y)]

        return new_data
